In [1]:
from Methods_xlWings import *
from Options_Math_Algebra import *
from Options_Math_Helpers import *
from Time_Stuff import *


import json
import numpy as np
import pandas as pd
import xlwings as xw

from websocket import create_connection

oma = OptionsMathAlgebra()
omh = OptionsMathHelpers()
ts  = TimeStuff()

In [2]:
deribit_url = 'wss://www.deribit.com/ws/api/v2' 

def deribit_request(method, params):
    msg = {'jsonrpc': 2.0,
           'id'     : 1,
           'method' : method,
           'params' : params
    }
    ws.send(json.dumps(msg))
    return json.loads(ws.recv())
    
def get_instruments(details):
    return deribit_request('public/get_instruments', details)['result']

def get_ticker(name):
    return deribit_request('public/ticker', {'instrument_name': name})['result']

In [3]:
def get_ticks(kind, url, instruments):

    rows = []    
    for inst in instruments:
        ticker = get_ticker(inst)
        rows.append(ticker)
       
    df = pd.DataFrame(rows)
    
    return df

In [4]:
def get_insts(kind, url):
    
    instruments = get_instruments({'currency': 'BTC',
                                   'kind'    : kind,
                                   'expired' : False
                                  })
    
    df = pd.DataFrame(instruments)
    
    return df

In [5]:
ws = create_connection(deribit_url)

dict_of_dfs = {}
for kind in ['future', 'option']:
    
    df_insts = get_insts(kind, deribit_url)
    df_ticks = get_ticks(kind, deribit_url, df_insts['instrument_name'].tolist())

    df = pd.merge(df_insts, df_ticks, on='instrument_name', how='outer')

    for col in ['stats', 'tick_size_steps']:
        df = flatten_dict_column(df, col, sep='_')

    if kind == 'option':
        for col in ['greeks', 'tick_size_steps_0']:
            df = flatten_dict_column(df, col, sep='_')
        
    printDFToXL('deribit.xlsx', kind + 's_raw', 'a1', df)    

    dict_of_dfs[kind] = df

ws.close()

print('done')

done


In [63]:
df = pd.concat([dict_of_dfs['future'], dict_of_dfs['option']], ignore_index=True)

In [65]:
from datetime import datetime, date, time, timedelta, timezone
from zoneinfo import ZoneInfo

df['current_time_utc']     = pd.Timestamp(datetime.now(timezone.utc))
df['current_time_nyc']     = df['current_time_utc'].dt.tz_convert('America/New_York')

df['expiration_time_utc'], = omh.to_arrays(df['expiration_timestamp'], dtype='datetime64[ms]')  

mask = df['expiration_time_utc'] > np.datetime64('2999-12-31')
df.loc[mask, 'expiration_time_utc'] = np.datetime64('NaT')

df['expiration_time_utc']  = df['expiration_time_utc'].dt.tz_localize("UTC")
df['expiration_time_nyc']  = df['expiration_time_utc'].dt.tz_convert('America/New_York')

df['years_to_expiry']      = (df['expiration_time_nyc'] - df['current_time_nyc']) / pd.Timedelta(days=1)
df['years_to_expiry']      = df['years_to_expiry'] / 365.25

df['settlement_time_nyc']  = df['expiration_time_nyc']
    
df['days_to_settlement']   = df['settlement_time_nyc'].dt.date - df['current_time_nyc'].dt.date 
df['days_to_settlement']   = pd.to_timedelta(df['days_to_settlement']) / pd.Timedelta(days=1)

df['years_to_settlement']  = df['days_to_settlement'] / 360   # money market

#df['days_thirty']          = ts.accrual_days_thirty(df['current_time_nyc'], df['settlement_date_nyc'])

In [66]:
df = df.sort_values(by=['years_to_expiry', 'strike', 'option_type'])

printDFToXL('deribit.xlsx', 'combo_with_time', 'a1', df)    

print('done')

done
